# LangGraph with Unity Catalog Functions - Tool Calling System

This code creates a **conversational AI** that can call **Unity Catalog functions** as tools, using LangGraph + Databricks.

## 🔧 **Setup & Configuration**
- **MLflow tracking**: `mlflow.langchain.autolog()` logs all LLM interactions
- **State management**: `State(MessagesState)` handles conversation history
- **LLM**: Databricks GPT OSS 120B model endpoint

## 🗃️ **Unity Catalog Integration**
```python
uc_tool_names = ("agents.main.*",)  # Pattern to match UC functions
tools = UCFunctionToolkit(function_names=list(uc_tool_names)).tools
```
- **UCFunctionToolkit**: Automatically discovers and loads functions from Unity Catalog
- **Pattern matching**: `"agents.main.*"` loads all functions from the `agents.main` schema
- **Auto-discovery**: No need to manually define tools - UC functions become AI tools

## 🤖 **LLM + Tools Setup**
```python
llm_with_tools = llm.bind_tools(tools)  # LLM gets access to UC functions
```
The LLM can now call any function from the Unity Catalog schema as needed.

## 🕸️ **Graph Workflow**


START → tool_calling_llm → [conditional routing] → tools → tool_calling_llm → END



- **tool_calling_llm**: LLM decides whether to use UC functions or respond directly
- **tools**: Executes the actual Unity Catalog functions
- **Conditional routing**: Automatically routes to tools when LLM needs them

## 🔄 **Execution Flow**
1. **User query**: *"Can you tell me more info about brittanyramos@example.org?"*
2. **LLM analysis**: Determines if UC functions can help with this email query
3. **Tool calling**: May call UC functions like user lookup, email validation, etc.
4. **Response**: Combines tool results with LLM reasoning

## 🎯 **Key Benefits**
- **Enterprise integration**: Direct access to company's UC functions
- **Zero setup**: No manual tool definition required
- **Scalable**: Automatically discovers new UC functions
- **Secure**: Uses Unity Catalog's permissions and governance
- **Production ready**: Enterprise-grade function execution

This pattern enables AI agents to use your organization's existing data functions and business logic stored in Unity Catalog.

In [0]:
%sql 
create catalog if not exists agents;
create schema if not exists agents.main;


## Get the customer details based on their email

Let's add a function to retrieve a customer detail based on their email.


In [0]:
%sql
CREATE OR REPLACE FUNCTION agents.main.get_customer_by_email(email_input STRING COMMENT 'customer email used to retrieve customer information')
RETURNS TABLE (
    customerID BIGINT,
    first_name STRING,
    last_name STRING,
    email_address STRING,
    phone_number STRING,
    address STRING,
    city STRING,
    state STRING,
    postal_zip_code STRING,
    country STRING,
    continent STRING,
    gender STRING
)
COMMENT 'Returns the customer record matching the provided email address. Includes its customerID, first_name, last_name and more.'
RETURN (
    SELECT * FROM samples.bakehouse.sales_customers
    WHERE email_address = email_input
    LIMIT 1
);

In [0]:
%sql SELECT * FROM agents.main.get_customer_by_email('brittanyramos@example.org');

In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks, UCFunctionToolkit
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


from langgraph.graph import MessagesState



import mlflow
mlflow.langchain.autolog()


class State(MessagesState):
    pass




llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")


uc_tool_names  = ("agents.main.*",)


tools  = UCFunctionToolkit(function_names=list(uc_tool_names)).tools


llm_with_tools = llm.bind_tools(tools)

def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

result = graph.invoke({"messages": [HumanMessage("Can you tell me more info about brittanyramos@example.org?")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")




In [0]:
result = graph.invoke({"messages": [HumanMessage("Which table are you using to get the user details")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")

In [0]:
result = graph.invoke({"messages": [HumanMessage("Tell me more about the user table")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")

In [0]:
result = graph.invoke({"messages": [HumanMessage("Show me only the metadata for the user table")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")